In [1]:
from utils.common import *

# Get Data

In [2]:
import urllib.request
import json


def retrieve(url):
    print(url)
    with urllib.request.urlopen(url) as response:
        html = response.read().decode("utf-8")
        data = json.loads(html)
    return data

In [3]:
def query_chart_data(begin_datetime, end_datetime, pair, period):
    begin, end = int(begin_datetime.timestamp()), int(end_datetime.timestamp())
    url = f'https://poloniex.com/public?command=returnChartData&currencyPair={pair}&start={begin}&end={end}&period={period}'
    return retrieve(url)


def query_trade_history(begin_datetime, end_datetime, pair):
    begin, begin = begin_datetime.timestamp(), end_datetime.timestamp()
    url = f'https://poloniex.com/public?command=returnTradeHistory&currencyPair={pair}&start={begin}&end={end}'
    return retrieve(url)

In [4]:
from dateutil.rrule import rrule, YEARLY, MONTHLY
from dateutil.relativedelta import relativedelta
from time import sleep


def retrieve_long_window(begin_time, end_time, pair, period):
    begin, end = str_time_to_datetime(begin_time, end_time)
    data = []
    for dt in rrule(YEARLY, dtstart=begin, until=end):
        dt2 = dt+relativedelta(years=1)
        data += query_chart_data(dt, dt2, pair, period)
        sleep(1)
    
    return data

In [5]:
def rename_date_to_timestamp(df):
    return df.rename(columns={'date': 'timestamp'})

In [6]:
def poloniex_data_to_df(data):
    df = pd.DataFrame(data)
    df = rename_date_to_timestamp(df).query('timestamp > 0')
    df = reset_index_hard(df)
    df['date'] = df['timestamp'].apply(timestamp_to_date)
    return df.drop_duplicates()

# Download Chart Data

In [7]:
from sqlite3 import connect

def create_table(table_name, conn):
    sql_create = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
            timestamp       BIGINT,
            period          STRING,
            open            DOUBLE,
            high            DOUBLE,
            low             DOUBLE,
            close           DOUBLE,
            volume          DOUBLE,
            quoteVolume     DOUBLE,
            weightedAverage DOUBLE,
            date            DATETIME,
            PRIMARY KEY (
                timestamp,
                period
            )
        );
    """

    conn.execute(sql_create);
    

def remove_duplicates_of_timestamp(df):
    data = df.copy()
    diffs = data['timestamp'].shift(-1) - data['timestamp']
    data['timestampDiff'] = diffs
    zero_diff = data.query('timestampDiff == 0')
    data_dropped_duplicates = reset_index_hard(df.drop(zero_diff.index))
    return data_dropped_duplicates

def write_sql(df, db, table_name):
    conn = connect(db)
    create_table(table_name, conn)
    
    key_cols = ['timestamp', 'period']
    old = read_table(table_name, db, conn)
    keys = old.append(df, sort=True)\
        .groupby(key_cols)\
        .count()\
        .query('close == 1')\
        .reset_index()[key_cols]
    not_in_db = df.merge(keys, on=key_cols, how='inner')
    not_in_db.to_sql(table_name, conn, if_exists='append', index=False)
    
    all_data = read_table(table_name, db, conn)
    all_data = remove_duplicates_of_timestamp(all_data)
    all_data.to_sql(table_name, conn, if_exists='replace', index=False)

In [8]:
def retrieve_and_save(pair, start_time, end_time, interval, interval_str):
    data = retrieve_long_window(start_time, end_time, pair, interval)
    df = poloniex_data_to_df(data)
    df['period'] = interval_str
    
    db_root = "D:\\Dropbox\\My work\\krypl-project\\sqlite\\ploniex-chart-data"
    db = f"{db_root}\\{pair}.db"
    write_sql(df, db, 'chart_data')

In [9]:
start_time, end_time = '2014-01-02 00:00:00', '2018-10-01 00:00:00'
interval, interval_str = 300, '5min'
pairs = [
    'USDT_BTC',
#     'BTC_ETH',
#     'BTC_LTC',
#     'BTC_XRP',
#     'BTC_XMR',
#     'BTC_STR'
]

In [12]:
for pair in pairs:
    print(pair, end='\t')
    retrieve_and_save(pair, start_time, end_time, interval, interval_str)
    print('done')

USDT_BTC	https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1388617200&end=1420153200&period=300


HTTPError: HTTP Error 403: Forbidden